In [220]:
from math import exp

In [221]:
def round_to_2(x):
    """
    Принимает число и возвращает результат его округления
    до 2 знаков после запятой.
    
    Аргументы:
        x: Число.
        
    Возвращаемое значение:
        Результат округления числа до 2 знаков после запятой.
    """
    
    return round(x, 2)

## Перцептрон

In [222]:
def sign(x):
    """
    Возвращает 1 если переданное численное значение больше 0,
    в противном случае возвращает -1.
    
    Аргументы:
        x: Численное значение.
        
    Возвращаемое значение:
        1, если x > 0, -1 в противном случае.
    """
    
    if x > 0:
        return 1
    
    return -1

def sigmoid(x):
    """
    Возвращает 1 / (1 + e^(-x)).
    
    Аргументы:
        x: Численное значение.
        
    Возвращаемое значение:
        1 / (1 + e^(-x))
    """
    
    return round_to_2(1 / (1 + exp(-x)))

def relu(x):
    """
    Возвращает наибольшее из чисел x и 0.
    
    Аргументы:
        x: Численное значение.
        
    Возвращаемое значение:
        Наибольшее из чисел x и 0
    """
    
    if x > 0:
        return x
    
    return 0

In [661]:
def perceptron_classify_solution(ws, xs, f=sign):
    """
    Вычисляет предсказания перцептрона с заданными параметрами для набора объектов.
    
    Аргументы:
        ws: Список весов дендритов перцептрона. 
            Вес на $i$-ой позиции соответствует весу $i$-го дендрита.
            Последний вес в списке соответствует весу $w_b$ искусственного дендрита, который
            отвечает за константную добавку к взвешенной сумме входящих в нейрон сигналов.
        xs: Список объектов. Каждый объект представляется в виде списка значений факторов.
            Каждый объект описывается одним и тем же набором факторов.
            Фактор под номером $i$ передаёт сигнал на $i$-ый дендрит нейрона.
            При этом в данных отсутствует искусственный фактор с константным значением $1$,
            который необходим для того, чтобы передать сигнал на соответствующий искусственный дендрит.
         f: Функция активации нейрона. По умолчанию задана как функция sign.
        
    Возвращаемое значение:
        Список предсказаний перцептрона для всех объектов.
        Предсказание для $i$-го объекта находится в списке на позиции $i$.
    """
    
    ans = []
    c = ws[len(ws) - 1]
    summ = 0
    for i in range(len(xs)):
        for x, w in zip (xs[i],ws):
            summ += x * w
            
        summ += c
        if summ > 0 and summ < 1:  
            f = sigmoid 
        ans.append(f(summ))    
        summ = 0
        
    
    return ans

In [439]:
def perceptron_classify_tests():
    example_1_ws = [5, -1, 3]
    example_1_xs = [[2, 3],
                    [-1, 4]]
    
    example_1_res = [1, -1]
    
    assert perceptron_classify_solution(example_1_ws, example_1_xs) == example_1_res
    
    example_2_ws = [10, -21, 47, 54, 11]
    example_2_xs = [[-5, 3, 9, -1],
                    [2, 42, -11, 17],
                    [-21, 7, 23, 8],
                    [24, -67, -24, 92]
                   ]
    
    example_2_res = [1, -1, 1, 1]
    
    assert perceptron_classify_solution(example_2_ws, example_2_xs) == example_2_res
    
    example_3_ws = [0.39, 0.74, 0.61, 0.37, 0.46, 0.3, 0.87]
    example_3_xs = [[0.38, -0.47, 0.61, -0.87, 0.41, -0.38],
                    [-0.31, 0.38, -0.74, 0.61, -0.38, 0.46],
                    [0.93, 0.6, -0.83, 0.4, -0.8, 0.75]
                   ]
    
    example_3_res = [0.69, 0.68, 0.76]
    
    assert perceptron_classify_solution(example_3_ws, example_3_xs, f=sigmoid) == example_3_res
    
    print('Все тесты прошли успешно!')

In [440]:
perceptron_classify_tests()

Все тесты прошли успешно!


In [419]:
def perceptron_train_solution(xs, ys, learning_rate=1, k=100):
    """
    Обучает перцептрон с функцией активации sign проводить классификацию переданного набора данных.
    В результате работы возвращает веса обученного перцептрона.
    
    Стартовые значения весов перцептрона задаются равными $0$.
    
    В процессе обучения используется два критерия, согласно которым происходит
    остановка обучения перцептрона:
    1. Если число итераций обучения превысило некоторую константу k, 
       то обучение перцептрона останавливается.
    2. Если значение метрики «среднее отклонение предсказанного значения от действительного» меньше или равно $0$,
       то обучение перцептрона останавливается.
    
    Аргументы:
        xs: Список объектов обучающей выборки. Каждый объект представляется в виде списка значений факторов.
            Каждый объект описывается одним и тем же набором факторов.
            Фактор под номером $i$ передаёт сигнал на $i$-ый дендрит нейрона.
            При этом в данных отсутствует искусственный фактор с константным значением $1$,
            который необходим для того, чтобы передать сигнал на соответствующий искусственный дендрит.
        ys: Список классов, которым принадлежат объекты из xs.
            Порядок классов соответствует порядку объектов в xs.
            Класс может быть либо $-1$, либо $1$.
        learning_rate: Константа скорости обучения, которая используется в правиле обновления весов перцептрона.
                       По умолчанию аргумент равен $1$.
         k: Число итераций, после которого необходимо прервать обучение перцептрона,
            если процесс не остановился естественным образом.
            По умолчанию аргумент равен $100$.
        
    Возвращаемое значение:
        Список весов дендритов перцептрона.
        Последний вес в списке соответствует весу $w_b$ искусственного дендрита, который
        отвечает за константную добавку к взвешенной сумме входящих в нейрон сигналов.
    """
    
    ws = [0] * (len(xs[0]) + 1)
    average_metric = 1
    constt = 0
    summ_y = 0
    
    while constt <= k and average_metric >= 0:
        for i in range(len(xs)):
            classify_solution = perceptron_classify_solution(ws, xs)
            y = (ys[i] - classify_solution[i])
            for j in range(len(xs[0])):
                ws[j] += learning_rate * y * xs[i][j]
            ws[len(ws) - 1] += learning_rate * y 
                
        for i in range(len(xs)):
            summ_y += abs(ys[i] - classify_solution[i])
            
        average_metric = summ_y / 2 * (len(ws) - 1)
        constt += 1
        
    
    return ws

In [420]:
def perceptron_train_test():
    example_1_xs = [
        [5, -2],
        [-2, -1],
        [3, 4],
        [-1, -2]
    ]
    example_1_ys = [1, -1, -1, 1]
    
    example_1_res = [8, -10, -2]
    
    assert perceptron_train_solution(example_1_xs, example_1_ys) == example_1_res
    
    example_2_xs = [
        [-5, 3, 9, -1],
        [2, 42, -11, 17],
        [-21, 7, 23, 8],
        [24, -67, -24, 92]
    ]
    example_2_ys = [1, 1, -1, -1]
    
    example_2_res = [26, 112, -26, -218, -4]
    
    assert perceptron_train_solution(example_2_xs, example_2_ys) == example_2_res
    
    example_3_xs = [
        [2, 3, 23, 2],
        [-1, 4, -2, 32],
        [23, -53, -11, 21],
        [11, -29, -53, -23],
        [-2, 9, 2, -3]
    ]
    example_3_ys = [-1, 1, 1, -1, -1]
    
    example_3_res = [18, -70, -16, 150, -4]
    
    assert perceptron_train_solution(example_3_xs, example_3_ys) == example_3_res
    
    print('Все тесты прошли успешно!')

In [421]:
perceptron_train_test()

Все тесты прошли успешно!


# Многослойный перцептрон

In [677]:
def multilayer_perceptron_solution(wss, fs, x):
    """
    Вычисляет предсказание многослойного перцептрона для конкретного объекта.
    
    Аргументы:
        wss: Список списков весов нейронов перцептрона.
        
             Длина внешнего списка равна числу слоёв нейрона.
             Каждый элемент внешнего списка — набор параметров нейронов конкретного слоя.
             $i$-ый элемент внешнего списка соответствует $i$-ому слою многослойного перцептрона.
             
             Параметры нейронов конкретного слоя тоже представлены в виде списка.
             Каждый элемент такого списка — набор параметров конкретного нейрона.
             
             Набор параметров конкретного нейрона тоже представлен в виде списка.
             Каждый элемент такого списка — вес соответствующего дендрита нейрона.
             Последний вес в списке соответствует весу $w_b$ искусственного дендрита,
             который отвечает за константную добавку к взвешенной сумме входящих в нейрон сигналов.
        
        fs: Список функций активации, которые используются в каждом из слоёв.
            $i$-ая функция активации используется в $i$-ом слое многослойного перцептрона.
        
         x: Объект, для которого необходимо сделать предсказание.
            Объект представляется в виде списка значений факторов.
            У объекта отсутствует искусственный фактор с константным значением $1$,
            который необходим для того, чтобы передать сигнал на соответствующий искусственный
            дендрит нейронов первого слоя.
        
    Возвращаемое значение:
        Предсказанное значение.
    """
    ns = []
    ns.append(x)
    for i in range(len(wss)):
        ans = []
        for j in range(len(wss[i])):
            a = perceptron_classify_solution(wss[i][j], ns, fs[i])
            ans.append(a[0])
            
        ns.clear()
        ns.append(ans)
        
    
    return ns[0][0]         

In [678]:
def multilayer_perceptron_tests():
    example_1_wss = [
        [[1, -1, 5]],
        [[2, 1],
         [2, -3]],
        [[1, -4, 1]]
    ]

    example_1_fs = [relu, relu, sigmoid]
    example_1_x = [-1, 1]
    
    example_1_res = 0.02
    
    assert multilayer_perceptron_solution(example_1_wss, example_1_fs, example_1_x) == example_1_res
    
    example_2_wss = [
        [[2, -3, 7], [-6, 5, 2]],
        [[2, 2, 1]]
    ]

    example_2_fs = [relu, relu]
    example_2_x = [-2, -2]
    
    example_2_res = 27
    
    assert multilayer_perceptron_solution(example_2_wss, example_2_fs, example_2_x) == example_2_res
    
    print('Все тесты прошли успешно!')

In [679]:
multilayer_perceptron_tests()

Все тесты прошли успешно!
